In [1]:
%load_ext autoreload
%autoreload 2

# MTBO Reizman Study

In [2]:
from summit.strategies import MTBO, Transform
from summit.benchmarks import ReizmanSuzukiEmulator
from summit.utils.dataset import DataSet
from summit.domain import *
import summit
import pathlib
import pandas as pd

## Create DataSet

In [5]:
# Path to data
summit_path = pathlib.Path(summit.__file__).parents[0]
data_path = summit_path / "benchmarks" / "experiment_emulator" / "data"

In [15]:
# Import Reizman-Suzuki Data
datasets = [DataSet.read_csv(data_path / f"reizman_suzuki_case{i}_train_test.csv") for i in range(1,5)]
for i, dataset in enumerate(datasets):
    dataset[('task', 'METADATA')] = i
datasets[0].head(5)

,catalyst,t_res,temperature,catalyst_loading,ton,yield,task
0,P1-L3,600,30,0.498,1.1,0.6,0
1,P1-L6,600,30,2.515,0.2,0.6,0
2,P1-L4,60,30,2.508,0.2,0.6,0
3,P1-L1,60,30,0.513,1.1,0.6,0
4,P1-L2,600,30,2.513,0.2,0.6,0


In [18]:
d = datasets[0]
d[['t_res', 'catalyst', 'temperature']]

,t_res,catalyst,temperature
0,600,P1-L3,30
1,600,P1-L6,30
2,60,P1-L4,30
3,60,P1-L1,30
4,600,P1-L2,30
...,...,...,...
91,189.7,P1-L6,110
92,600,P1-L4,110
93,600,P1-L3,65.3
94,600,P2-L1,30


In [9]:
domain = Domain()
domain += CategoricalVariable('base', 'Organic Base', levels=['DBU', 'BMTG', 'TEA'])
domain += ContinuousVariable('yield', '', bounds=[0,100], is_objective=True)
transform = Transform(domain)
X = DataSet([['DBU', 20], ['BMTG', 30], ['BMTG', 40], ['TEA', 10]], columns=['base', 'yield'])
inputs, outputs = transform.transform_inputs_outputs(X, categorical_method='one-hot')

In [10]:
inputs

""
0
1
2
3
